In [11]:
%load_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [12]:
%autoreload 2

In [13]:
import pandas as pd
import numpy as np
from scipy import stats
from datetime import datetime

''' Sn properties - Masses, Ratios, Cup Configurations, ... '''
from sn_config import *
from collections import Counter
import pandas as pd
import numpy as np

'''Classes for Reading in the Data and applying Internal Normalisation'''
from nu_data_reduction import NU_data_read, int_norm, evaluation, Neptune_data_read
from parameter import *
from dspike_formulas import *



In [175]:
path = "/Volumes/friebelm/PhD/NU Plasma/Measurements/2017-09-20_DS_mixtures/"
files_start = 8383
files_end = 8516

In [176]:
# Isotopes used for Spike Inversion [[denominator isotope], nominator isotopes, .. ]
spikeSn_ls = [["117"],["118", "122", "124"]]
# Dictionary with pure DS composition
spike_dict_117_122_meas = {"118": 0.0102951622334242, "119": 0.0090731830784630, "120" : 0.0338645175569080, "122" : 0.8068023572167630, "124" : 0.0114846216495240} # DS compositon 1ppm - 15.09.17
# Dictionary with Standard composition
meas_dict = { "118" : 3.15777585757689, "119" : 1.11972979701600, "120" : 4.25050484928267, "122" : 0.60392336483548, "124" : 0.75469249128349} # mean of 1ppm NIST Std used for calib-15.09.17
# Denominator isotope in ratios for DS & Std composition
meas_denom_117 = "117"

# Load compositions in for DS inversion
Sn_meas_obj = load_ratio_dict(meas_dict_NIST_117, meas_denom_117)
spike_obj = load_ratio_dict(spike_dict_117_122_meas, meas_denom_117)

In [209]:
# cup configuration
#cup_config = cycle_Sb
Sn_isotopes = ["117", "118", "119", "120", "122", "124"]
cup_config = cycles_spike

# Mass Range of cup configuration
mass_range = cycle_Sn_spike_mass_range

# Isotopes used for Interference correction
corr_isotopes_2 = {"Te" : "126", "Xe" : "129"}
#corr_isotopes_2 = {}
#corr_isotopes_Sb = {"Te": "125"}

denom_isotope = "117"
Sn_monitor = ["126"]

#def eval_iso_list(isotopes_list, denom_isotope, monitor_iso):
#    isotope_den = denom_isotope
#    isotopes_list.remove(isotope_den)
#    for i in range(len(monitor_iso)):
#        isotopes_list.append(monitor_iso[i])
#    isotopes_list.sort()
#   return isotopes_list

#isotope_ls = eval_iso_list(Sn_isotopes, denom_isotope, Sn_monitor)

#data_sample_column = [(i + "/" + denom_isotope) for i in isotope_ls]    
isotopes = [["118", "122", "124"]]

# Amount of interations for estimation of instrumental and natural fractionation
loop_nat = 3
loop_ins = 6
# assumended instrumental and natural fractionation
start_nat = -0.1
start_ins = -2
# isotope ratio used for calculation of instrumental and natural fractionation [denominator[x, y, z]] e.g. [117,[118, 122, 124]] x = 118/117
inv_iso_ratio = 'x'



# Interference_corr on the denominator isotope
isotope_denom_corr = False
# background correction
blk_corr = True

In [210]:
isotopes

[['118', '122', '124']]

In [179]:
columns_1 = ["cycle", "sample", "date", "H8 (1)", "H7 (1)", "H6 (1)", "H5 (1)", "H4 (1)", "H3 (1)", "H2 (1)", "H1 (1)", "Ax (1)", "L1 (1)", "L2 (1)", "L3 (1)", "L4 (1)"]
df_all = pd.DataFrame()
blk_ls = []
sample_ls = []
files_1 = range(files_start, files_end, 1)
#files_1 = range(5552,5623,1) + range(5632,5729, 1)

for sample in files_1:
    
    df = NU_data_read(path, sample, cup_config)
    cycles = range(1, len(df.data_read(sample).index)+1)
    df_zero = df.data_zero_corr(sample)
    df_zero_1 = pd.DataFrame(df_zero["cycle1"])
    sample_name = df.extract_metadata(sample, "Sample Name")
    date = df.extract_metadata(sample, "Date")
    starttime = df.extract_metadata(sample, "Start Time")
    df_zero_1["date"] = datetime.strptime(date+starttime, '%d/%m/%Y%H:%M')
    df_zero_1["sample"] = sample_name
    df_zero_1["cycle"] = df_zero_1.index.values
    df_zero_1.index = df_zero_1["date"]
    df_zero_1 = df_zero_1[columns_1]
    df_all = df_all.append(df_zero_1, ignore_index = True)
    
    if sample_name == "blank sol" or sample_name == "wash" or sample_name == "wash clean" or sample_name == "Teflon blk":
        blk_ls.append(sample)
    elif sample_name == "SQ" or sample_name == "Teflon blk" or sample_name == "4ml HDPE blank":
        None
    else:
        sample_ls.append(sample)

df_all.set_index("date", inplace = True)

#for x in [123]:
#    sample_ls.remove(x)
#blk_ls = blk_ls[0::5]
#blk_ls.remove(5234)

In [180]:
df_all

,cycle,sample,H8 (1),H7 (1),H6 (1),H5 (1),H4 (1),H3 (1),H2 (1),H1 (1),Ax (1),L1 (1),L2 (1),L3 (1),L4 (1)
date,,,,,,,,,,,,,,,
2017-09-21 14:50:00,1,blank sol,0.011327,0.013713,0.016516,0.001898,0.000698,0.000653,0.000676,0.000391,0.000778,0.002825,0.000239,0.000692,0.000238
2017-09-21 14:50:00,2,blank sol,0.011268,0.013695,0.016473,0.001906,0.000679,0.000659,0.000672,0.000382,0.000785,0.002781,0.000256,0.000693,0.000221
2017-09-21 14:50:00,3,blank sol,0.011297,0.013676,0.016506,0.001910,0.000701,0.000657,0.000664,0.000360,0.000801,0.002811,0.000240,0.000661,0.000226
2017-09-21 14:50:00,4,blank sol,0.011287,0.013719,0.016464,0.001894,0.000672,0.000645,0.000700,0.000374,0.000780,0.002801,0.000235,0.000685,0.000247
2017-09-21 14:50:00,5,blank sol,0.011269,0.013682,0.016416,0.001892,0.000655,0.000677,0.000690,0.000362,0.000773,0.002768,0.000261,0.000677,0.000230
2017-09-21 14:50:00,6,blank sol,0.011299,0.013751,0.016432,0.001887,0.000673,0.000673,0.000679,0.000401,0.000781,0.002868,0.000269,0.000744,0.000247
2017-09-21 14:50:00,7,blank sol,0.011338,0.013773,0.016463,0.001902,0.000673,0.000685,0.000671,0.000383,0.000798,0.002908,0.000267,0.000761,0.000248
2017-09-21 14:50:00,8,blank sol,0.011314,0.013752,0.016503,0.001902,0.000674,0.000656,0.000668,0.000386,0.000780,0.002794,0.000258,0.000675,0.000219
2017-09-21 14:50:00,9,blank sol,0.011304,0.013716,0.016420,0.001886,0.000687,0.000650,0.000679,0.000389,0.000792,0.002770,0.000231,0.000664,0.000216


In [211]:
files_1 = range(files_start, files_end, 1)
df_raw_signals = pd.DataFrame()
df_DS_inv= pd.DataFrame()

for sample in sample_ls:
    df = NU_data_read(path, sample, cup_config)
    cycles = range(1, len(df.data_read(sample).index)+1)
    
    # arbitrary blank positions - check blk position before and after sample 
    blk1 = [item for item in blk_ls if item < sample]
    blk2 = [item for item in blk_ls if item > sample]
    blk1 = blk1[-1]
    blk2 = blk2[0]
    blk_corr_sample = True #if True blank corr_activated
    
    # blank measurement surrounds sample
    #if ((sample-1) in blk_ls) and ((sample+1) in blk_ls):
    #    blk1 = sample-1
    #    blk2 = sample+1
    #    blk_corr_sample = True
    #else:
    #    blk_corr_sample = False
    
    # Background correction
    if (blk_corr == True) and (blk_corr_sample == True):
        # Reads signals of Blank measurements
        blk_1 = NU_data_read(path, blk1, cup_config)
        blk_2 = NU_data_read(path, blk2, cup_config)
        # Substracts baselines of signals
        df_zero = df.data_zero_corr(sample)
        df_bgd_1 = blk_1.data_zero_corr(blk1)
        df_bgd_2 = blk_2.data_zero_corr(blk2)   
        
        # Defines evaluation class with all evaluation methods present
        new_corr = evaluation(df_zero, cycles, isotopes, cup_config, database, mass_range, corr_isotopes_2 , denom_corr_ratio)
        # Outlier correction on signals of blank measurements
        df_bgd_1 = new_corr.mad_outlier_rejection_dict(df_bgd_1)
        df_bgd_2 = new_corr.mad_outlier_rejection_dict(df_bgd_2)
        # Background substraction of blank signals
        new_corr.data_bgd_corr(df_bgd_1, df_bgd_2)
    
    # No background correction
    else:
        # Substracts baselines of signals
        df_zero = df.data_zero_corr(sample)
        # Defines evaluation class with all evaluation methods present
        new_corr = evaluation(df_zero, cycles, isotopes, cup_config, database, mass_range, corr_isotopes_2 , denom_corr_ratio)
    
    # Gives zero_corrected Signals back & writes it in a dataframe
    data_sample_raw = new_corr.raw_signals(denom_isotope)
    data_sample_raw = pd.DataFrame.from_dict(data_sample_raw, orient = 'index')
    data_sample_raw = new_corr.mad_outlier_rejection(data_sample_raw)
    
    #Get raw ratios
    data_sample = new_corr.raw_ratios(denom_isotope)
    # raw ratios from dataframe in dictionary for data handling
    data_sample = pd.DataFrame.from_dict(data_sample, orient = 'index')
    # First DS-inversion without interference correction
    data_spike_obj = calc_dspike_sample(Sn_meas_obj, data_sample, spike_obj, Sn_mass_obj, spikeSn_ls , denom_isotope)
    data_spike_calc = data_spike_obj.dspike_corr(loop_nat, loop_ins, start_nat, start_ins, inv_iso_ratio)
    # Beta for interference correction
    beta = data_spike_calc["frac_ins_x2.5"]
    # Interference correction
    spike_corr = new_corr.norm_beta_to_raw("Sn", "117", beta)
    # interference corrected raw ratios
    data_corr_raw = pd.DataFrame.from_dict(spike_corr, orient = 'index')
    # Second DS-inversion
    data_spike_obj = calc_dspike_sample(Sn_meas_obj, data_corr_raw, spike_obj, Sn_mass_obj, spikeSn_ls , denom_isotope)
    data_spike_calc_2 = data_spike_obj.dspike_corr(loop_nat, loop_ins, start_nat, start_ins, inv_iso_ratio)
    #Calculation of eSn
    data_spike_calc_2["eSn_118"] = ((data_spike_calc_2["Nr2:x"]/data_spike_calc_2["n0:x"])-1)*10000
    data_spike_calc_2["eSn_122"] = ((data_spike_calc_2["Nr2:y"]/data_spike_calc_2["n0:y"])-1)*10000
    data_spike_calc_2["eSn_124"] = ((data_spike_calc_2["Nr2:z"]/data_spike_calc_2["n0:z"])-1)*10000
    data_spike_calc_2["eSn_122_118"] = (((data_spike_calc_2["Nr2:y"]/data_spike_calc_2["Nr2:x"])/(data_spike_calc_2["n0:y"]/data_spike_calc_2["n0:x"]))-1)*10000
    data_spike_calc_2["eSn_124_118"] = (((data_spike_calc_2["Nr2:z"]/data_spike_calc_2["Nr2:x"])/(data_spike_calc_2["n0:z"]/data_spike_calc_2["n0:x"]))-1)*10000
    data_spike_calc_2["fsam"] = 1/((data_spike_calc_2["Mr2.5:x"]*0.07669971 - 0.24220048)/(0.00525699 - data_spike_calc_2["Mr2.5:x"]*0.51062751)+1)
    data_spike_calc_2["fspike"] = 1-data_spike_calc_2["fsam"]
    data_spike_calc_2["Q"] = data_spike_calc_2["fsam"]/data_spike_calc_2["fspike"]
    #Outlier rejection
    data_spike_calc_2 = new_corr.mad_outlier_rejection(data_spike_calc_2)
    
    # Extraction of Sample Name, Date, Measurement time and add it to Dataframe
    sample_name = df.extract_metadata(sample, "Sample Name")
    date = df.extract_metadata(sample, "Date")
    starttime = df.extract_metadata(sample, "Start Time")
    data_spike_calc_2["date"] = datetime.strptime(date+starttime, '%d/%m/%Y%H:%M')
    data_spike_calc_2["sample"] = sample_name
    data_spike_calc_2["cycle"] = data_sample.index.values
    
    data_sample_raw["cycle"] = data_sample_raw.index.values
    data_spike_calc_2 = data_spike_calc_2.merge(data_sample_raw)
    
    # Add each sample in for loop to a dataframe
    df_DS_inv = df_DS_inv.append(data_spike_calc_2)

In [212]:
df_DS_inv = df_DS_inv.set_index("date")
columns = ["cycle", "sample", "eSn_118", "eSn_122", "eSn_124", "eSn_122_118","eSn_124_118", "Q"]
columns.extend(df_DS_inv.drop(["cycle", "sample", "eSn_118", "eSn_122", "eSn_124", "eSn_122_118", "eSn_124_118", "Q"], axis=1).columns)
df_DS_inv_sub = pd.DataFrame(df_DS_inv, columns = columns)

#df_internal_sub["Cd/Sn"] = (df_internal_norm["111"]/12.722)/(df_internal_norm["118"]/24.223)
#df_internal_sub["Te/Sn"] = (df_internal_norm["125"]/7.0509)/(df_internal_norm["118"]/24.223)

#df_internal_sub["Te/Sn"] = (df_internal_norm["126"]/18.8066)/(df_internal_norm["118"]/24.223)
#df_internal_sub = df_internal_sub[df_internal_sub["118"] >= 7]
#df_external_sub.to_csv(path_sav + "Sn_H9_L4_internal_corr_all.csv")
df_DS_inv_sub 

,cycle,sample,eSn_118,eSn_122,eSn_124,eSn_122_118,eSn_124_118,Q,n0:x,n0:y,...,Nr2:x,Nr2:y,Nr2:z,frac_nat_x2,fsam,fspike,122,124,118,117
date,,,,,,,,,,,,,,,,,,,,,
2017-09-21 14:55:00,1,NIST + DS opt. 0.6-0.4,-1.252347,-6.165054,-8.561465,-4.913323,-7.310034,1.544769,3.157776,0.603923,...,3.157380,0.603551,0.754046,-0.014723,0.607037,0.392963,8.985967,1.817759,6.674693,10.913953
2017-09-21 14:55:00,2,NIST + DS opt. 0.6-0.4,-1.487049,-7.320109,-10.165272,-5.833928,-8.679514,1.544870,3.157776,0.603923,...,3.157306,0.603481,0.753925,-0.017483,0.607053,0.392947,9.274815,1.876143,6.889254,11.263996
2017-09-21 14:55:00,3,NIST + DS opt. 0.6-0.4,-1.446367,-7.119905,-9.887292,-5.674359,-8.442146,NaN,3.157776,0.603923,...,3.157319,0.603493,0.753946,-0.017005,NaN,NaN,9.671461,1.956694,7.185631,11.746737
2017-09-21 14:55:00,4,NIST + DS opt. 0.6-0.4,-1.169809,-5.758830,-7.997402,-4.589558,-6.828392,1.544841,3.157776,0.603923,...,3.157406,0.603576,0.754089,-0.013753,0.607048,0.392952,9.790554,1.980606,7.273049,11.892101
2017-09-21 14:55:00,5,NIST + DS opt. 0.6-0.4,-1.276711,-6.284965,-8.727966,-5.008893,-7.452207,1.544945,3.157776,0.603923,...,3.157373,0.603544,0.754034,-0.015010,0.607064,0.392936,9.703110,1.962961,7.208272,11.785434
2017-09-21 14:55:00,6,NIST + DS opt. 0.6-0.4,-1.436312,-7.070424,-9.818588,-5.634921,-8.383480,NaN,3.157776,0.603923,...,3.157322,0.603496,0.753951,-0.016886,NaN,NaN,9.667698,1.956723,7.185220,11.741510
2017-09-21 14:55:00,7,NIST + DS opt. 0.6-0.4,-1.355223,-6.671358,-9.264483,-5.316856,-7.910332,1.544923,3.157776,0.603923,...,3.157348,0.603520,0.753993,-0.015933,0.607061,0.392939,9.605133,1.943075,7.135234,11.666062
2017-09-21 14:55:00,8,NIST + DS opt. 0.6-0.4,-1.306237,-6.430278,-8.929739,-5.124711,-7.624498,1.544793,3.157776,0.603923,...,3.157363,0.603535,0.754019,-0.015357,0.607041,0.392959,9.463980,1.914395,7.030160,11.495165
2017-09-21 14:55:00,9,NIST + DS opt. 0.6-0.4,-1.005286,-4.949065,-6.872973,-3.944175,-5.868276,1.544947,3.157776,0.603923,...,3.157458,0.603624,0.754174,-0.011819,0.607064,0.392936,9.263322,1.874092,6.882483,11.253202


In [213]:
def SE(x):
    SE = np.std(x) / x.count()
    return SE

def SD2(x):
    SD2 = 2 * np.std(x)
    return SD2

df_DS_inv_mean = df_DS_inv_sub.drop("cycle", axis=1).groupby(df_DS_inv_sub.index).mean()
df_DS_inv_mean_sd = df_DS_inv_sub.drop("cycle", axis=1).groupby(df_DS_inv_sub.index).agg([np.mean, SD2])
df_DS_inv_mean["sample"] = df_DS_inv_sub[~df_DS_inv_sub.index.duplicated(keep='first')]["sample"]
df_DS_inv_mean_sd["sample"] = df_DS_inv_sub[~df_DS_inv_sub.index.duplicated(keep='first')]["sample"]
#df_internal_norm_mean_sd = df_internal_norm_mean_sd[1:8]
#df_internal_norm_mean = df_internal_norm_mean[df_internal_norm_mean["118"] >= 7]
df_DS_inv_mean_sd
#epsilon = epsilon.drop(["118"], axis = 1)
#plotplot = df_internal_norm_mean[(df_internal_norm_mean["125"] < 0.00010)  & (df_internal_norm_mean["124/120"] > 0.17745)] 
#plot(plotplot["125"], plotplot["124/120"], "ro")

eSn_118             eSn_122             eSn_124  \
                         mean       SD2      mean       SD2      mean   
date                                                                    
2017-09-21 14:55:00 -1.288552  0.399225 -6.343204  1.964805 -8.808808   
2017-09-21 15:11:00 -1.328835  0.371022 -6.541459  1.826018 -9.084093   
2017-09-21 15:27:00 -1.295506  0.484600 -6.377409  2.385047 -8.856290   
2017-09-21 15:43:00 -1.304475  0.491086 -6.421546  2.416966 -8.917575   
2017-09-21 16:00:00 -1.223752  0.404388 -6.024284  1.990253 -8.365973   
2017-09-21 16:16:00 -1.245100  0.577787 -6.129310  2.843688 -8.511779   
2017-09-21 16:32:00 -1.277981  0.383649 -6.291179  1.888108 -8.736570   
2017-09-21 16:48:00 -1.171857  0.526444 -5.768845  2.591083 -8.011263   
2017-09-21 17:04:00 -1.317309  0.452323 -6.484719  2.226061 -9.005298   
2017-09-21 17:20:00 -0.536394  0.397147 -2.640896  1.955104 -3.667662   
2017-09-21 17:37:00 -1.285892  0.465323 -6.330099  2.290151 -8.790601   
2017-09-21 17:53:00 -1.283439  0.526096 -6.318011  2.589193 -8.773807   
2017-09-21 18:10:00 -1.277792  0.291808 -6.290263  1.436152 -8.735309   
2017-09-21 18:26:00 -1.262840  0.432412 -6.216654  2.128142 -8.633083   
2017-09-21 18:48:00 -1.301658  0.462235 -6.407689  2.274917 -8.898337   
2017-09-21 19:04:00 -1.239668  0.326086 -6.102628  1.604861 -8.474766   
2017-09-21 19:21:00 -1.267484  0.456700 -6.239504  2.247661 -8.664808   
2017-09-21 19:37:00 -1.290402  0.416800 -6.352304  2.051279 -8.821441   
2017-09-21 19:53:00 -1.286471  0.488077 -6.332941  2.402082 -8.794544   
2017-09-21 20:09:00 -1.228894  0.292094 -6.049610  1.437534 -8.401152   
2017-09-21 20:25:00 -1.241459  0.385675 -6.111432  1.898183 -8.486984   
2017-09-21 20:41:00 -1.261672  0.257603 -6.210932  1.267809 -8.625158   
2017-09-21 20:57:00 -1.047188  0.540505 -5.155235  2.660297 -7.159213   
2017-09-21 21:13:00 -1.173882  0.409460 -5.778836  2.015291 -8.025153   
2017-09-21 21:30:00 -0.452100  0.232418 -2.225936  1.144194 -3.091405   
2017-09-21 21:46:00 -1.204308  0.382169 -5.928592  1.880895 -8.233103   
2017-09-21 22:02:00 -1.198728  0.332158 -5.901136  1.634793 -8.194984   
2017-09-21 22:18:00 -1.250153  0.458419 -6.154208  2.256138 -8.546371   
2017-09-21 22:34:00 -1.166188  0.351082 -5.740977  1.727974 -7.972592   
2017-09-21 22:50:00 -1.249646  0.433932 -6.151717  2.135573 -8.542916   
...                       ...       ...       ...       ...       ...   
2017-09-22 00:43:00 -1.215393  0.436255 -5.983135  2.146995 -8.308831   
2017-09-22 00:59:00 -1.227521  0.449778 -6.042825  2.213655 -8.391712   
2017-09-22 01:15:00 -1.152593  0.404797 -5.674057  1.992354 -7.879662   
2017-09-22 01:31:00 -1.250784  0.478388 -6.157309  2.354525 -8.550673   
2017-09-22 01:47:00 -0.499232  0.340232 -2.457955  1.674923 -3.413611   
2017-09-22 02:04:00 -1.206411  0.465687 -5.938921  2.291931 -8.247434   
2017-09-22 02:20:00 -1.291795  0.512388 -6.359138  2.521622 -8.830915   
2017-09-22 02:36:00 -1.240083  0.447033 -6.104649  2.200110 -8.477558   
2017-09-22 02:52:00 -1.270865  0.364501 -6.256164  1.793922 -8.687954   
2017-09-22 03:08:00 -1.255885  0.340827 -6.182441  1.677424 -8.585588   
2017-09-22 03:24:00 -1.275820  0.590475 -6.280498  2.906009 -8.721707   
2017-09-22 03:40:00 -1.237578  0.455970 -6.092321  2.244105 -8.460438   
2017-09-22 03:56:00 -1.206887  0.375166 -5.941286  1.846444 -8.250729   
2017-09-22 04:13:00 -1.264118  0.458197 -6.222937  2.255046 -8.641805   
2017-09-22 04:29:00 -1.220831  0.393743 -6.009911  1.937873 -8.346016   
2017-09-22 04:45:00 -1.235291  0.380516 -6.081081  1.872760 -8.444841   
2017-09-22 05:01:00 -1.140582  0.297853 -5.614963  1.466001 -7.797618   
2017-09-22 05:17:00 -1.217462  0.554355 -5.993290  2.728381 -8.322913   
2017-09-22 05:33:00 -1.057455  0.659677 -5.205734  3.246650 -7.229312   
2017-09-22 05:49:00 -1.193896  0.453867 -5.877333  2.233818 -8.161917   
2017-09-22 06:05:00 -0.371976  0.301413 -1.831459  1.483909 -2.543563   
2017-09

In [214]:
df_DS_inv_mean_sample_sd = df_DS_inv_mean.groupby(df_DS_inv_mean["sample"]).agg([np.mean, SD2])
df_DS_inv_mean_sample_sd

eSn_118             eSn_122             eSn_124  \
                             mean       SD2      mean       SD2      mean   
sample                                                                      
NIST + DS 0.4-0.6       -1.257380  0.092190 -6.189792  0.453717 -8.595793   
NIST + DS 0.5-0.5       -1.249876  0.075641 -6.152845  0.372253 -8.544480   
NIST + DS 0.7-0.3       -1.210633  0.084071 -5.959710  0.413743 -8.276303   
NIST + DS 0.8-0.2       -1.107273  0.110983 -5.450968  0.546272 -7.569863   
NIST + DS opt. 0.6-0.4  -1.239819  0.083716 -6.103354  0.412008 -8.475763   
SPEX1 + DS opt. 0.6-0.4 -0.464926  0.122837 -2.289062  0.604729 -3.179060   

                                  eSn_122_118           eSn_124_118            \
                              SD2        mean       SD2        mean       SD2   
sample                                                                          
NIST + DS 0.4-0.6        0.630001   -4.933046  0.361619   -7.339356  0.537949   
NIST + DS 0.5-0.5        0.516876   -4.903602  0.296695   -7.295545  0.441358   
NIST + DS 0.7-0.3        0.574485   -4.749671  0.329763   -7.066554  0.490549   
NIST + DS 0.8-0.2        0.758563   -4.344205  0.435374   -6.463350  0.647706   
NIST + DS opt. 0.6-0.4   0.572085   -4.864155  0.328378   -7.236867  0.488496   
SPEX1 + DS opt. 0.6-0.4  0.839813   -1.824232  0.481941   -2.714278  0.717049   

                           ...       fspike                  122            \
                           ...         mean       SD2       mean       SD2   
sample                     ...                                               
NIST + DS 0.4-0.6          ...     0.590534  0.000019  19.656090  0.519876   
NIST + DS 0.5-0.5          ...     0.494490  0.000016  13.450436  0.400962   
NIST + DS 0.7-0.3          ...     0.293513  0.000008   6.314838  0.167434   
NIST + DS 0.8-0.2          ...     0.194281  0.000007   4.193504  0.094885   
NIST + DS opt. 0.6-0.4     ...     0.392933  0.000009   8.948948  0.349651   
SPEX1 + DS opt. 0.6-0.4    ...     0.431158  0.000014   9.324581  0.215559   

                              124                 118                  117  \
                             mean       SD2      mean       SD2       mean   
sample                                                                       
NIST + DS 0.4-0.6        2.090292  0.054947  7.255146  0.193245  23.288617   
NIST + DS 0.5-0.5        1.955301  0.058003  7.007554  0.209937  16.096637   
NIST + DS 0.7-0.3        1.798396  0.047550  6.712862  0.183090   7.823147   
NIST + DS 0.8-0.2        1.753038  0.039556  6.634155  0.150672   5.368908   
NIST + DS opt. 0.6-0.4   1.810734  0.070661  6.645978  0.259716  10.865039   
SPEX1 + DS opt. 0.6-0.4  1.664753  0.038502  6.057865  0.141068  11.251475   

                                   
                              SD2  
sample                             
NIST + DS 0.4-0.6        0.622348  
NIST + DS 0.5-0.5        0.483554  
NIST + DS 0.7-0.3        0.213660  
NIST + DS 0.8-0.2        0.122075  
NIST + DS opt. 0.6-0.4   0.424898  
SPEX1 + DS opt. 0.6-0.4  0.262110  

[6 rows x 648 columns]

In [215]:
std_used = "NIST + DS opt. 0.6-0.4 "
sub_index = df_DS_inv_mean[(df_DS_inv_mean["sample"].shift(1) == df_DS_inv_mean["sample"].shift(-1)) & (df_DS_inv_mean["sample"].shift(1) == std_used)].index
corr_values = df_DS_inv_mean[df_DS_inv_mean["124"] >= 0.1]
corr_values_2 = corr_values[corr_values["sample"] == std_used]
#corr_values_2 = corr_values[corr_values["sample"] == std_used]
#corr_values = df_internal_norm_mean
sample = corr_values["sample"]
sample_2 = corr_values_2["sample"]
std_mean = corr_values[(corr_values["sample"] == std_used)].mean()
new = corr_values[corr_values.columns[corr_values.dtypes != 'object']]
new_2 = corr_values_2[corr_values_2.columns[corr_values_2.dtypes != 'object']]

#std_mean["112/120", "114/120", "115/120", "117/120", "118/120", "119/120", "122/120", "124/120"] = [0.029823, 0.020189, 0.010372, 0.235299, 0.742923, 0.263443, 0.142078, 0.177546]
#print std_mean
#epsilon =  ((new / std_mean)-1)*10000
#new["sample"] = sample
epsilon_NIST =  ((new - ((new.shift(1) + new.shift(-1))/2)))
epsilon_2 = ((new_2 - ((new_2.shift(1) + new_2.shift(-1))/2)))
epsilon_NIST["sample"] = sample
epsilon_2["sample"] = sample_2

epsilon_NIST = epsilon_NIST[new.index.isin(sub_index)]
#epsilon_NIST= epsilon_NIST[epsilon_NIST["sample"].str.contains("opt")]
#epsilon_NIST= epsilon_NIST[epsilon_NIST["sample"] != std_used]
epsilon_2 = epsilon_2[epsilon_2["sample"] == std_used]
#epsilon_2 = epsilon_2[epsilon_2["sample"] == std_used]
epsilon_comp = epsilon_NIST.append(epsilon_2, ignore_index=False)
#epsilon_comp.index = epsilon_comp["Date"]
#epsilon_comp.columns
epsilon_comp = epsilon_comp.sort_index(axis=0)

In [216]:
epsilon_comp[epsilon_comp["sample"] == "NIST + DS opt. 0.6-0.4 "]

,eSn_118,eSn_122,eSn_124,eSn_122_118,eSn_124_118,Q,n0:x,n0:y,n0:z,N0:x,...,Nr2:y,Nr2:z,frac_nat_x2,fsam,fspike,122,124,118,117,sample
date,,,,,,,,,,,,,,,,,,,,,
2017-09-21 14:55:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NIST + DS opt. 0.6-0.4
2017-09-21 15:27:00,-0.039354,-0.193665,-0.268899,-0.154357,-0.229614,9.795585e-06,0.0,0.0,0.0,0.0,...,-1.169587e-05,-2.029363e-05,-0.000463,1.512657e-06,-1.512657e-06,-0.130872,-0.026532,-0.097348,-0.159032,NIST + DS opt. 0.6-0.4
2017-09-21 16:00:00,0.062991,0.310010,0.430457,0.247084,0.367562,7.684861e-06,0.0,0.0,0.0,0.0,...,1.872223e-05,3.248628e-05,0.000741,1.186476e-06,-1.186476e-06,0.059026,0.012030,0.043667,0.071277,NIST + DS opt. 0.6-0.4
2017-09-21 16:32:00,-0.007450,-0.036677,-0.050935,-0.029230,-0.043489,1.061796e-07,0.0,0.0,0.0,0.0,...,-2.215008e-06,-3.844009e-06,-0.000088,1.650380e-08,-1.650380e-08,-0.001381,-0.000332,-0.000862,-0.001252,NIST + DS opt. 0.6-0.4
2017-09-21 17:04:00,-0.035373,-0.174081,-0.241712,-0.138746,-0.206397,3.320899e-06,0.0,0.0,0.0,0.0,...,-1.051314e-05,-1.824186e-05,-0.000416,5.126131e-07,-5.126131e-07,0.046939,0.009533,0.034786,0.056625,NIST + DS opt. 0.6-0.4
2017-09-21 17:37:00,0.014482,0.071266,0.098951,0.056802,0.084495,-3.336616e-06,0.0,0.0,0.0,0.0,...,4.303932e-06,7.467774e-06,0.000170,-5.151042e-07,5.151042e-07,-0.050923,-0.010378,-0.038228,-0.061991,NIST + DS opt. 0.6-0.4
2017-09-21 17:53:00,-0.001597,-0.007830,-0.010852,-0.006247,-0.009275,-6.148030e-06,0.0,0.0,0.0,0.0,...,-4.728690e-07,-8.189594e-07,-0.000019,-9.493411e-07,9.493411e-07,0.024109,0.005005,0.018564,0.029548,NIST + DS opt. 0.6-0.4
2017-09-21 17:53:00,-0.001597,-0.007830,-0.010852,-0.006247,-0.009275,-6.148030e-06,0.0,0.0,0.0,0.0,...,-4.728690e-07,-8.189594e-07,-0.000019,-9.493411e-07,9.493411e-07,0.024109,0.005005,0.018564,0.029548,NIST + DS opt. 0.6-0.4
2017-09-21 18:10:00,-0.004652,-0.022931,-0.031864,-0.018269,-0.027198,1.547803e-05,0.0,0.0,0.0,0.0,...,-1.384838e-06,-2.404764e-06,-0.000055,2.389922e-06,-2.389922e-06,-0.006170,-0.001296,-0.004945,-0.007788,NIST + DS opt. 0.6-0.4


In [217]:
def SD2(x):
    SD2 = np.nanstd(x, ddof=1) * 2
    return SD2

epsilon = epsilon_comp[["eSn_118", "eSn_122", "eSn_124", "eSn_122_118", "eSn_124_118", "sample"]]
#epsilon = epsilon.drop(["118"], axis = 1)
epsilon_sum = np.round(epsilon.groupby("sample").agg([np.mean, SD2]), decimals=2)
epsilon_sum

eSn_118       eSn_122       eSn_124       eSn_122_118  \
                           mean   SD2    mean   SD2    mean   SD2        mean   
sample                                                                          
NIST + DS 0.4-0.6         -0.01  0.08   -0.07  0.38   -0.10  0.52       -0.06   
NIST + DS 0.5-0.5         -0.01  0.07   -0.04  0.33   -0.06  0.46       -0.03   
NIST + DS 0.7-0.3          0.02  0.09    0.11  0.43    0.15  0.60        0.08   
NIST + DS 0.8-0.2          0.13  0.07    0.65  0.35    0.91  0.49        0.52   
NIST + DS opt. 0.6-0.4    -0.00  0.10   -0.02  0.47   -0.02  0.65       -0.01   
SPEX1 + DS opt. 0.6-0.4    0.77  0.09    3.77  0.44    5.23  0.62        3.00   

                              eSn_124_118        
                          SD2        mean   SD2  
sample                                           
NIST + DS 0.4-0.6        0.30       -0.08  0.44  
NIST + DS 0.5-0.5        0.26       -0.05  0.39  
NIST + DS 0.7-0.3        0.34        0.12  0.51  
NIST + DS 0.8-0.2        0.28        0.77  0.42  
NIST + DS opt. 0.6-0.4   0.37       -0.02  0.55  
SPEX1 + DS opt. 0.6-0.4  0.35        4.47  0.53

In [196]:
df_all.to_csv(path + "Signals_zero.csv")

In [218]:
df_DS_inv_mean_sd.to_csv(path + "DS_inv_bgd_corr_126Te.csv")

In [219]:
df_DS_inv_sub.to_csv(path + "DS_inv_bdg_corr_all_126Te.csv")

In [68]:
# For one datapoint
files_1 = 7208

df = NU_data_read(path, sample, cup_config)
cycles = range(1, len(df.data_read(sample).index)+1)

# arbitrary blank positions
#blk1 = [item for item in blk_ls if item < sample]
#blk2 = [item for item in blk_ls if item > sample]
#blk1 = blk1[-1]
#blk2 = blk2[0]
#blk_corr_sample = True
    
if (blk_corr == True) and (blk_corr_sample == True):
    blk_1 = NU_data_read(path, blk1, cup_config)
    blk_2 = NU_data_read(path, blk2, cup_config)
    df_zero = df.data_zero_corr(sample)
    df_bgd_1 = blk_1.data_zero_corr(blk1)
    df_bgd_2 = blk_2.data_zero_corr(blk2)   
        
    new_corr = evaluation(df_zero, cycles, isotopes, cup_config, database, mass_range, corr_isotopes_2 , denom_corr_ratio)
    df_bgd_1 = new_corr.mad_outlier_rejection_dict(df_bgd_1)
    df_bgd_2 = new_corr.mad_outlier_rejection_dict(df_bgd_2)
    df_bgd_corr = new_corr.data_bgd_corr(df_bgd_1, df_bgd_2)
    #new_corr.line2_corr(df_bgd_corr, "119")
else:
    df_zero = df.data_zero_corr(sample)
    new_corr = evaluation(df_zero, cycles, isotopes, cup_config, database, mass_range, corr_isotopes_2 , denom_corr_ratio)
    #new_corr.line2_corr(df_zero, "119")

data_sample = new_corr.raw_ratios(denom_isotope)
data_sample = pd.DataFrame.from_dict(data_sample, orient = 'index')
df_zero

{'cycle1': {'Ax (1)': Counter({1: 0.00013836589999999954,
           2: 0.00012571850000000009,
           3: 0.00014383660000000069,
           4: 0.00014579099999999928,
           5: 0.00014788939999999841,
           6: 0.00014835819999999868,
           7: 0.00015420509999999887,
           8: 0.00015245509999999886,
           9: -4.0447400000000883e-05,
           10: 0.00014881189999999857,
           11: 0.00012035759999999909,
           12: 0.00014608449999999919,
           13: 0.00012900490000000014,
           14: 0.0003078537999999997,
           15: 0.00014112969999999919,
           16: 0.00014536620000000014,
           17: 0.00013632270000000016,
           18: 0.00014228219999999972,
           19: 0.00012344829999999876,
           20: 0.00012890919999999917}),
  'H1 (1)': Counter({1: 9.6512528109999991,
           2: 9.5990288110000002,
           3: 9.5930372110000004,
           4: 9.5689413110000014,
           5: 9.4110702110000002,
           6: 9.63697051100

In [69]:
#Spike Calculation uncorrected
data_spike_obj = calc_dspike_sample(Sn_meas_obj, data_sample, spike_obj, Sn_mass_obj, spikeSn_ls , denom_isotope)
data_spike_calc = data_spike_obj.dspike_corr(3, 6, -0.1, -2, 'x')
beta = data_spike_calc["frac_ins_x2.5"]
beta

0    -1.510864
1    -1.508789
2    -1.508917
3    -1.506711
4    -1.505094
5    -1.511240
6    -1.512046
7    -1.509094
8     1.503680
9    -1.522139
10   -1.505025
11   -1.506463
12   -1.507148
13   -1.841712
14   -1.503643
15   -1.508317
16   -1.505471
17   -1.505653
18   -1.501966
19   -1.499314
Name: frac_ins_x2.5, dtype: float64

In [70]:
data_spike_calc

,n0:x,n0:y,n0:z,N0:x,N0:y,N0:z,a_nat0,b_nat0,c_nat0,m0.0:x,...,e_nat2,f_nat2,g_nat2,a_ins2,b_ins2,c_ins2,Nr2:x,Nr2:y,Nr2:z,frac_nat_x2
0,3.157775,0.60392,0.754685,3.15509,0.601396,0.750308,0.325585,1.387874,-1.111603,0.619120,...,-4.404098,1.735338,-0.293320,0.251475,0.240988,-0.185533,3.157274,0.603449,0.753868,-0.018641
1,3.157775,0.60392,0.754685,3.15509,0.601396,0.750308,0.325585,1.387874,-1.111603,0.619114,...,-4.403804,1.735305,-0.293300,0.251451,0.240961,-0.185511,3.157212,0.603391,0.753767,-0.020938
2,3.157775,0.60392,0.754685,3.15509,0.601396,0.750308,0.325585,1.387874,-1.111603,0.619126,...,-4.403658,1.735289,-0.293290,0.251439,0.240954,-0.185506,3.157182,0.603362,0.753717,-0.022083
3,3.157775,0.60392,0.754685,3.15509,0.601396,0.750308,0.325585,1.387874,-1.111603,0.619114,...,-4.403766,1.735301,-0.293297,0.251448,0.240956,-0.185507,3.157204,0.603383,0.753754,-0.021240
4,3.157775,0.60392,0.754685,3.15509,0.601396,0.750308,0.325585,1.387874,-1.111603,0.619121,...,-4.403263,1.735245,-0.293264,0.251407,0.240918,-0.185476,3.157099,0.603284,0.753582,-0.025160
5,3.157775,0.60392,0.754685,3.15509,0.601396,0.750308,0.325585,1.387874,-1.111603,0.619135,...,-4.403640,1.735287,-0.293289,0.251438,0.240958,-0.185509,3.157178,0.603358,0.753711,-0.022219
6,3.157775,0.60392,0.754685,3.15509,0.601396,0.750308,0.325585,1.387874,-1.111603,0.619148,...,-4.403778,1.735302,-0.293298,0.251450,0.240974,-0.185522,3.157207,0.603385,0.753758,-0.021144
7,3.157775,0.60392,0.754685,3.15509,0.601396,0.750308,0.325585,1.387874,-1.111603,0.619166,...,-4.403734,1.735297,-0.293295,0.251447,0.240975,-0.185523,3.157198,0.603377,0.753743,-0.021482
8,3.157775,0.60392,0.754685,3.15509,0.601396,0.750308,0.325585,1.387874,-1.111603,0.579143,...,-5.048140,1.804461,-0.335064,0.305595,0.271258,-0.210512,3.288068,0.736609,0.994117,4.753254
9,3.157775,0.60392,0.754685,3.15509,0.601396,0.750308,0.325585,1.387874,-1.111603,0.619386,...,-4.399025,1.734778,-0.292982,0.251067,0.240709,-0.185304,3.156210,0.602448,0.752131,-0.058280


In [71]:
data_corr = {}
data_corr_raw = {}
cycle_no = 20
cycles = range(0, len(data_sample.index))

spike_corr = new_corr.norm_beta_to_raw("Sn", "117", beta)
    
    #data_sample = pd.DataFrame.from_dict(data_sample, orient = 'index')


In [72]:
spike_corr

{1: {'118': 0.61912006113889029,
  '122': 0.81813878936573747,
  '124': 0.16732120064277978},
 2: {'118': 0.61911425168525391,
  '122': 0.81805517367790703,
  '124': 0.1672851940216227},
 3: {'118': 0.61912628974194117,
  '122': 0.81805195573888878,
  '124': 0.1672793221099298},
 4: {'118': 0.61911404608763743,
  '122': 0.81798069012325936,
  '124': 0.16726362958883415},
 5: {'118': 0.61912080426918348,
  '122': 0.81790354598387227,
  '124': 0.16722271311813908},
 6: {'118': 0.61913509399426914,
  '122': 0.81813101822741197,
  '124': 0.16729994458236438},
 7: {'118': 0.61914757992317704,
  '122': 0.81816441621570202,
  '124': 0.16731911851154876},
 8: {'118': 0.61916617794426432,
  '122': 0.81805887151409529,
  '124': 0.16729603930839582},
 9: {'118': 0.5791433497747327,
  '122': 0.74595684179471722,
  '124': 0.16880522537091386},
 10: {'118': 0.6193856353460705,
  '122': 0.8182938239108356,
  '124': 0.16717424799163497},
 11: {'118': 0.61913683769205263,
  '122': 0.81790057198446653,


In [73]:
data_corr_raw = pd.DataFrame.from_dict(spike_corr, orient = 'index')

In [75]:
data_corr_raw

,122,124,118
1,0.818139,0.167321,0.619120
2,0.818055,0.167285,0.619114
3,0.818052,0.167279,0.619126
4,0.817981,0.167264,0.619114
5,0.817904,0.167223,0.619121
6,0.818131,0.167300,0.619135
7,0.818164,0.167319,0.619148
8,0.818059,0.167296,0.619166
9,0.745957,0.168805,0.579143
10,0.818294,0.167174,0.619386


In [79]:
data_spike_obj = calc_dspike_sample(Sn_meas_obj, data_corr_raw, spike_obj, Sn_mass_obj, spikeSn_ls , denom_isotope)
data_spike_calc_2 = data_spike_obj.dspike_corr(3, 6, -0.1, -2.2, 'x')
data_spike_calc_2["eSn"] = ((data_spike_calc_2["Nr2:x"]/data_spike_calc_2["n0:x"])-1)*10000
data_spike_calc_2 = new_corr.mad_outlier_rejection(data_spike_calc_2)

In [80]:
data_spike_calc_2

,n0:x,n0:y,n0:z,N0:x,N0:y,N0:z,a_nat0,b_nat0,c_nat0,m0.0:x,...,f_nat2,g_nat2,a_ins2,b_ins2,c_ins2,Nr2:x,Nr2:y,Nr2:z,frac_nat_x2,eSn
0,3.157775,0.60392,0.754685,3.15509,0.601396,0.750308,0.325585,1.387874,-1.111603,0.619120,...,1.735336,-0.293319,0.251475,0.240987,-0.185532,3.157272,0.603447,0.753864,-0.018726,-1.592763
1,3.157775,0.60392,0.754685,3.15509,0.601396,0.750308,0.325585,1.387874,-1.111603,0.619114,...,1.735305,-0.293300,0.251451,0.240961,-0.185511,3.157213,0.603391,0.753768,-0.020908,-1.778381
2,3.157775,0.60392,0.754685,3.15509,0.601396,0.750308,0.325585,1.387874,-1.111603,0.619126,...,1.735287,-0.293289,0.251438,0.240953,-0.185505,3.157179,0.603359,0.753712,-0.022195,-1.887829
3,3.157775,0.60392,0.754685,3.15509,0.601396,0.750308,0.325585,1.387874,-1.111603,0.619114,...,1.735298,-0.293296,0.251446,0.240954,-0.185505,3.157199,0.603378,0.753745,-0.021441,-1.823652
4,3.157775,0.60392,0.754685,3.15509,0.601396,0.750308,0.325585,1.387874,-1.111603,0.619121,...,1.735246,-0.293264,0.251407,0.240918,-0.185476,3.157099,0.603284,0.753582,-0.025148,-2.138903
5,3.157775,0.60392,0.754685,3.15509,0.601396,0.750308,0.325585,1.387874,-1.111603,0.619135,...,1.735285,-0.293288,0.251437,0.240957,-0.185508,3.157175,0.603355,0.753705,-0.022338,-1.899962
6,3.157775,0.60392,0.754685,3.15509,0.601396,0.750308,0.325585,1.387874,-1.111603,0.619148,...,1.735302,-0.293298,0.251450,0.240974,-0.185522,3.157207,0.603385,0.753757,-0.021152,-1.799092
7,3.157775,0.60392,0.754685,3.15509,0.601396,0.750308,0.325585,1.387874,-1.111603,0.619166,...,1.735297,-0.293295,0.251446,0.240975,-0.185522,3.157196,0.603375,0.753740,-0.021542,-1.832220
8,3.157775,0.60392,0.754685,3.15509,0.601396,0.750308,0.325585,1.387874,-1.111603,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,3.157775,0.60392,0.754685,3.15509,0.601396,0.750308,0.325585,1.387874,-1.111603,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [22]:
data_spike_calc_2.to_csv(path+"test_2.csv")

eSn           n0:x          n0:y           n0:z  \
                            mean  SD2      mean  SD2     mean  SD2      mean   
sample                                                                         
DS-NIST Mix 1ppm opti. -1.803065  0.0  3.157775  0.0  0.60392  0.0  0.754685   

                                N0:x      ...  frac_nat_x2            122  \
                        SD2     mean  SD2 ...         mean  SD2      mean   
sample                                    ...                               
DS-NIST Mix 1ppm opti.  0.0  3.15509  0.0 ...    -0.021199  0.0  9.636478   

                                  124            118             117       
                        SD2      mean  SD2      mean  SD2       mean  SD2  
sample                                                                     
DS-NIST Mix 1ppm opti.  0.0  1.971941  0.0  7.302154  0.0  11.789357  0.0  

[1 rows x 634 columns]